# Peta Sebaran Covid-19 di Indonesia

Notebook ini akan mengambil data kasus Covid-19 per provinsi di Indonesia, menggabungkannya dengan data koordinat provinsi, dan memvisualisasikannya dalam bentuk peta interaktif menggunakan Folium.


In [1]:
import requests
import pandas as pd
import folium
from bs4 import BeautifulSoup

# URL sumber data
url_covid = "https://www.worldometers.info/coronavirus/country/indonesia/"  # Gantilah dengan sumber yang valid
url_gps = "https://raw.githubusercontent.com/LintangWisesa/Indonesia-Covid19-Maps/master/data/gps_indonesia.json"


In [2]:
# Step 1: Web Scraping Data Covid-19 per Provinsi
response = requests.get(url_covid)
soup = BeautifulSoup(response.text, 'html.parser')

# Jika web scraping tidak memungkinkan, gunakan data dummy
data_covid = pd.DataFrame({
    "Provinsi": ["DKI Jakarta", "Jawa Barat", "Jawa Tengah", "Jawa Timur", "Bali"],
    "Total Kasus": [5000, 4000, 3500, 4500, 1500],
    "Sembuh": [3000, 2500, 2000, 2700, 1000],
    "Meninggal": [500, 400, 350, 450, 100]
})

data_covid


,Provinsi,Total Kasus,Sembuh,Meninggal
0,DKI Jakarta,5000,3000,500
1,Jawa Barat,4000,2500,400
2,Jawa Tengah,3500,2000,350
3,Jawa Timur,4500,2700,450
4,Bali,1500,1000,100


In [3]:
# Step 2: Mendapatkan Data Lokasi (Latitude & Longitude)
response_gps = requests.get(url_gps)
data_gps = response_gps.json()
df_gps = pd.DataFrame(data_gps)

df_gps.head()


,Provinsi,latitude,longitude,logo
0,Aceh,4.695135,96.749399,https://upload.wikimedia.org/wikipedia/commons...
1,Bali,-8.409518,115.188916,https://4.bp.blogspot.com/-ELlrLdH0frM/WSz4Ajq...
2,Bangka Belitung,-2.741051,106.440587,https://upload.wikimedia.org/wikipedia/commons...
3,Banten,-6.405817,106.064018,https://upload.wikimedia.org/wikipedia/commons...
4,Bengkulu,-3.577847,102.346388,https://upload.wikimedia.org/wikipedia/commons...


In [4]:
# Step 3: Merge Data Covid dengan Koordinat
df_merged = pd.merge(data_covid, df_gps, left_on="Provinsi", right_on="lokasi", how="left")

df_merged.head()


KeyError: 'lokasi'

In [5]:
# Step 4: Membuat Peta Interaktif
peta = folium.Map(location=[-2.5489, 118.0149], zoom_start=5)

for _, row in df_merged.iterrows():
    folium.Marker(
        location=[row["lat"], row["lon"]],
        popup=f"{row['Provinsi']}<br>Total Kasus: {row['Total Kasus']}<br>Sembuh: {row['Sembuh']}<br>Meninggal: {row['Meninggal']}",
        icon=folium.Icon(color="red" if row["Total Kasus"] > 3000 else "blue")
    ).add_to(peta)

# Simpan peta sebagai file HTML
peta.save("peta_covid_indonesia.html")

peta  # Menampilkan peta dalam notebook


NameError: name 'df_merged' is not defined